## Problem Statement

You need to build a model that is able to classify customer complaints based on the products/services. By doing so, you can segregate these tickets into their relevant categories and, therefore, help in the quick resolution of the issue.

You will be doing topic modelling on the <b>.json</b> data provided by the company. Since this data is not labelled, you need to apply NMF to analyse patterns and classify tickets into the following five clusters based on their products/services:

* Credit card / Prepaid card

* Bank account services

* Theft/Dispute reporting

* Mortgages/loans

* Others


With the help of topic modelling, you will be able to map each ticket onto its respective department/category. You can then use this data to train any supervised model such as logistic regression, decision tree or random forest. Using this trained model, you can classify any new customer complaint support ticket into its relevant department.

## Pipelines that needs to be performed:

You need to perform the following eight major tasks to complete the assignment:

1.  Data loading

2. Text preprocessing

3. Exploratory data analysis (EDA)

4. Feature extraction

5. Topic modelling

6. Model building using supervised learning

7. Model training and evaluation

8. Model inference

## Importing the necessary libraries

In [4]:
import json
import numpy as np
import pandas as pd
import re, nltk, spacy, string
import en_core_web_sm
from tqdm import tqdm
nlp = en_core_web_sm.load()
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from plotly.offline import plot
import plotly.graph_objects as go
import plotly.express as px

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from pprint import pprint

# hide warnings
import warnings
warnings.filterwarnings('ignore')



In [5]:
tqdm.pandas()

In [6]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag

#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger_eng')

In [7]:
# change the display properties of pandas to max
pd.set_option('display.max_colwidth', 100)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

## Loading the data

The data is in JSON format and we need to convert it to a dataframe.

In [9]:
# Opening JSON file
f = open('complaints-2021-05-14_08_16.json')

# returns JSON object as
# a dictionary
data = json.load(f)
df=pd.json_normalize(data)

In [10]:
df.head(5)

,_index,_type,_id,_score,_source.tags,_source.zip_code,_source.complaint_id,_source.issue,_source.date_received,_source.state,...,_source.company_response,_source.company,_source.submitted_via,_source.date_sent_to_company,_source.company_public_response,_source.sub_product,_source.timely,_source.complaint_what_happened,_source.sub_issue,_source.consumer_consent_provided
0,complaint-public-v2,complaint,3211475,0.0,None,90301,3211475,Attempts to collect debt not owed,2019-04-13T12:00:00-05:00,CA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-13T12:00:00-05:00,None,Credit card debt,Yes,,Debt is not yours,Consent not provided
1,complaint-public-v2,complaint,3229299,0.0,Servicemember,319XX,3229299,Written notification about debt,2019-05-01T12:00:00-05:00,GA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-05-01T12:00:00-05:00,None,Credit card debt,Yes,Good morning my name is XXXX XXXX and I appreciate it if you could help me put a stop to Chase B...,Didn't receive enough information to verify debt,Consent provided
2,complaint-public-v2,complaint,3199379,0.0,None,77069,3199379,"Other features, terms, or problems",2019-04-02T12:00:00-05:00,TX,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-02T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,I upgraded my XXXX XXXX card in XX/XX/2018 and was told by the agent who did the upgrade my anni...,Problem with rewards from credit card,Consent provided
3,complaint-public-v2,complaint,2673060,0.0,None,48066,2673060,Trouble during payment process,2017-09-13T12:00:00-05:00,MI,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2017-09-14T12:00:00-05:00,None,Conventional home mortgage,Yes,,None,Consent not provided
4,complaint-public-v2,complaint,3203545,0.0,None,10473,3203545,Fees or interest,2019-04-05T12:00:00-05:00,NY,...,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2019-04-05T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,,Charged too much interest,N/A


In [11]:
df.shape

(78313, 22)

## Data preparation

In [13]:
# Inspect the dataframe to understand the given data.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78313 entries, 0 to 78312
Data columns (total 22 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   _index                             78313 non-null  object 
 1   _type                              78313 non-null  object 
 2   _id                                78313 non-null  object 
 3   _score                             78313 non-null  float64
 4   _source.tags                       10900 non-null  object 
 5   _source.zip_code                   71556 non-null  object 
 6   _source.complaint_id               78313 non-null  object 
 7   _source.issue                      78313 non-null  object 
 8   _source.date_received              78313 non-null  object 
 9   _source.state                      76322 non-null  object 
 10  _source.consumer_disputed          78313 non-null  object 
 11  _source.product                    78313 non-null  obj

In [14]:
#print the column names
df.columns

Index(['_index', '_type', '_id', '_score', '_source.tags', '_source.zip_code',
       '_source.complaint_id', '_source.issue', '_source.date_received',
       '_source.state', '_source.consumer_disputed', '_source.product',
       '_source.company_response', '_source.company', '_source.submitted_via',
       '_source.date_sent_to_company', '_source.company_public_response',
       '_source.sub_product', '_source.timely',
       '_source.complaint_what_happened', '_source.sub_issue',
       '_source.consumer_consent_provided'],
      dtype='object')

In [15]:
#Assign new column names
df.rename(columns={'_index':'index',
  '_type':'type',
  '_id':'id',
  '_score':'score',
  '_source.tags':'tags',
  '_source.zip_code':'zip_code',
 '_source.complaint_id':'complaint_id',
 '_source.issue':'issue',
 '_source.date_received':'date_received',
 '_source.state':'state',
 '_source.consumer_disputed':'consumer_disputed',
 '_source.product':'product',
 '_source.company_response':'company_response',
 '_source.company':'company',
 '_source.submitted_via':'submitted_via',
 '_source.date_sent_to_company':'date_sent_to_company',
 '_source.company_public_response':'company_public_response',
 '_source.sub_product':'sub_product',
 '_source.timely':'timely',
 '_source.complaint_what_happened':'complaint_what_happened',
 '_source.sub_issue':'sub_issue',
 '_source.consumer_consent_provided':'consumer_consent_provided'},inplace=True)

In [16]:
#Checking nulls values in each columns
df.isnull().sum()

index                            0
type                             0
id                               0
score                            0
tags                         67413
zip_code                      6757
complaint_id                     0
issue                            0
date_received                    0
state                         1991
consumer_disputed                0
product                          0
company_response                 0
company                          0
submitted_via                    0
date_sent_to_company             0
company_public_response      78309
sub_product                  10571
timely                           0
complaint_what_happened          0
sub_issue                    46297
consumer_consent_provided     1008
dtype: int64

In [17]:
# Checking rows where complaint_what_happened is blank
#df[df['complaint_what_happened']=='']

In [18]:
#Assign nan in place of blanks in the complaints column
df['complaint_what_happened'].replace(r'^\s*$', np.nan, regex=True, inplace=True)

In [19]:
#df[df['complaint_what_happened'].isna()]

In [20]:
df.isnull().sum()

index                            0
type                             0
id                               0
score                            0
tags                         67413
zip_code                      6757
complaint_id                     0
issue                            0
date_received                    0
state                         1991
consumer_disputed                0
product                          0
company_response                 0
company                          0
submitted_via                    0
date_sent_to_company             0
company_public_response      78309
sub_product                  10571
timely                           0
complaint_what_happened      57241
sub_issue                    46297
consumer_consent_provided     1008
dtype: int64

In [21]:
#Remove all rows where complaints column is nan
df.dropna(subset=['complaint_what_happened'],inplace=True)

In [22]:
df.isnull().sum()

index                            0
type                             0
id                               0
score                            0
tags                         17256
zip_code                      4645
complaint_id                     0
issue                            0
date_received                    0
state                          143
consumer_disputed                0
product                          0
company_response                 0
company                          0
submitted_via                    0
date_sent_to_company             0
company_public_response      21070
sub_product                   2109
timely                           0
complaint_what_happened          0
sub_issue                     8176
consumer_consent_provided        0
dtype: int64

## Prepare the text for topic modeling

Once you have removed all the blank complaints, you need to:

* Make the text lowercase
* Remove text in square brackets
* Remove punctuation
* Remove words containing numbers


Once you have done these cleaning operations you need to perform the following:
* Lemmatize the texts
* Extract the POS tags of the lemmatized text and remove all the words which have tags other than NN[tag == "NN"].


In [24]:
'''
def find_pattern(text, patterns):
    if re.search(patterns, text):
        return re.search(patterns, text)
    else:
        return 'Not Found!'
'''

"\ndef find_pattern(text, patterns):\n    if re.search(patterns, text):\n        return re.search(patterns, text)\n    else:\n        return 'Not Found!'\n"

In [25]:
#pattern = '\[\w+\]'
#pattern  = '\[[^\]]+\]'
#pattern  = '[^\w\s]'

In [26]:
#df['comp'] = df['complaint_what_happened'].apply(lambda x: find_pattern(x,pattern))

In [27]:
#df['comp'].value_counts()

In [28]:
#text = df['complaint_what_happened'][df['comp'] != 'Not Found!']

In [29]:
#text

In [30]:
'''
for i in text.index:
    print(i,'\n')
    print(df['complaint_what_happened'][i])
    print('*'*100)
'''   

"\nfor i in text.index:\n    print(i,'\n')\n    print(df['complaint_what_happened'][i])\n    print('*'*100)\n"

In [31]:
def clean_text(text):
    #Making text Lower case 
    text = text.lower()

    #Removing text in Square brackets []
    text = re.sub(r'\[[^\]]+\]',' ',text)
    #Removing Puntuations
    text = re.sub(r'[^\w\s/]',' ',text)

    # words containg numbers 
    text = re.sub(r'\b[a-zA-Z]*\d+[a-zA-Z]+\b|\b[a-zA-Z]+\d+\w*\b',' ',text)

    return text

In [32]:
'''
def clean_texts1(text):
    #Make the text lowercase
    text=text.lower()

    #Remove text in square brackets
    text=re.sub(r'','',text)

    #Remove punctuation
    text=re.sub(r'[%s]%re.escape(string.punctuation)','',text)

    #Remove words containing numbers
    text = re.sub(r'\w*\d\w*', '', text)

    return text
'''    

"\ndef clean_texts1(text):\n    #Make the text lowercase\n    text=text.lower()\n\n    #Remove text in square brackets\n    text=re.sub(r'','',text)\n\n    #Remove punctuation\n    text=re.sub(r'[%s]%re.escape(string.punctuation)','',text)\n\n    #Remove words containing numbers\n    text = re.sub(r'\\w*\\d\\w*', '', text)\n\n    return text\n"

In [33]:
'''
def clean_text2(text):
  text=text.lower()  #convert to lower case
  text=re.sub(r'^\[[\w\s]\]+$',' ',text) #Remove text in square brackets
  text=re.sub(r'[^\w\s]',' ',text) #Remove punctuation
  text=re.sub(r'^[a-zA-Z]\d+\w*$',' ',text) #Remove words with numbers
  return text
'''  

"\ndef clean_text2(text):\n  text=text.lower()  #convert to lower case\n  text=re.sub(r'^\\[[\\w\\s]\\]+$',' ',text) #Remove text in square brackets\n  text=re.sub(r'[^\\w\\s]',' ',text) #Remove punctuation\n  text=re.sub(r'^[a-zA-Z]\\d+\\w*$',' ',text) #Remove words with numbers\n  return text\n"

In [34]:
df['complaint_what_happened_cleaned'] = df['complaint_what_happened'].progress_apply(lambda x : clean_text(x))
#df['complaint_what_happened_cleaned1'] = df['complaint_what_happened'].apply(lambda x : clean_texts1(x))
#df['complaint_what_happened_cleaned2'] = df['complaint_what_happened'].apply(lambda x : clean_text2(x))


00%|██████████████████████████████████████████████████████████████████████████| 21072/21072 [00:04<00:00, 4881.77it/s]

In [35]:
df[['complaint_what_happened','complaint_what_happened_cleaned']]
#df[['complaint_what_happened','complaint_what_happened_cleaned','complaint_what_happened_cleaned1','complaint_what_happened_cleaned2']]

,complaint_what_happened,complaint_what_happened_cleaned
1,Good morning my name is XXXX XXXX and I appreciate it if you could help me put a stop to Chase B...,good morning my name is xxxx xxxx and i appreciate it if you could help me put a stop to chase b...
2,I upgraded my XXXX XXXX card in XX/XX/2018 and was told by the agent who did the upgrade my anni...,i upgraded my xxxx xxxx card in xx/xx/2018 and was told by the agent who did the upgrade my anni...
10,"Chase Card was reported on XX/XX/2019. However, fraudulent application have been submitted my id...",chase card was reported on xx/xx/2019 however fraudulent application have been submitted my id...
11,"On XX/XX/2018, while trying to book a XXXX XXXX ticket, I came across an offer for {$300.00} t...",on xx/xx/2018 while trying to book a xxxx xxxx ticket i came across an offer for 300 00 t...
14,my grand son give me check for {$1600.00} i deposit it into my chase account after fund clear my...,my grand son give me check for 1600 00 i deposit it into my chase account after fund clear my...
...,...,...
78303,"After being a Chase Card customer for well over a decade, was offered multiple solicitations for...",after being a chase card customer for well over a decade was offered multiple solicitations for...
78309,"On Wednesday, XX/XX/XXXX I called Chas, my XXXX XXXX Visa Credit Card provider, and asked how to...",on wednesday xx/xx/xxxx i called chas my xxxx xxxx visa credit card provider and asked how to...
78310,I am not familiar with XXXX pay and did not understand the great risk this provides to consumers...,i am not familiar with xxxx pay and did not understand the great risk this provides to consumers...
78311,"I have had flawless credit for 30 yrs. I've had Chase credit cards, "" Chase Freedom '' specifica...",i have had flawless credit for 30 yrs i ve had chase credit cards chase freedom specifica...


In [36]:
# Write your function here to clean the text and remove all the unnecessary elements.


In [37]:
#Write your function to Lemmatize the texts


In [38]:
from nltk.stem import WordNetLemmatizer

In [39]:
lemmatizer = WordNetLemmatizer()

In [40]:
#text = df['complaint_what_happened_cleaned'][1]

In [41]:
'''
words = []
lemm = []
for word in text.split():
    words.append(word)
    lemma = lemmatizer.lemmatize(word)
    lemm.append(lemma)
    #df1 = pd.DataFrame(data={'Word': word, 'lemmatized': lemma})
'''

"\nwords = []\nlemm = []\nfor word in text.split():\n    words.append(word)\n    lemma = lemmatizer.lemmatize(word)\n    lemm.append(lemma)\n    #df1 = pd.DataFrame(data={'Word': word, 'lemmatized': lemma})\n"

In [42]:
'''
df1 = pd.DataFrame(data={'Word': words, 'lemmatized': lemm})
df1[df1['Word'] != df1['lemmatized']]
'''

"\ndf1 = pd.DataFrame(data={'Word': words, 'lemmatized': lemm})\ndf1[df1['Word'] != df1['lemmatized']]\n"

In [43]:
def lemmatize(text):
    # Remove words inside square brackets
    #text = re.sub(r'\[\w+\]', '', text)
    words = word_tokenize(text)
    # Lemmatize each word
    #lemmatized_text = ' '.join(lemmatizer.lemmatize(word) for word in text.split())
    lemmatized_text = ' '.join(lemmatizer.lemmatize(word) for word in words)
    return lemmatized_text

In [44]:
df['complaint_what_happened_lemma'] = df['complaint_what_happened_cleaned'].progress_apply(lemmatize)


00%|███████████████████████████████████████████████████████████████████████████| 21072/21072 [00:56<00:00, 373.25it/s]

In [45]:
df[['complaint_what_happened_cleaned','complaint_what_happened_lemma']]

,complaint_what_happened_cleaned,complaint_what_happened_lemma
1,good morning my name is xxxx xxxx and i appreciate it if you could help me put a stop to chase b...,good morning my name is xxxx xxxx and i appreciate it if you could help me put a stop to chase b...
2,i upgraded my xxxx xxxx card in xx/xx/2018 and was told by the agent who did the upgrade my anni...,i upgraded my xxxx xxxx card in xx/xx/2018 and wa told by the agent who did the upgrade my anniv...
10,chase card was reported on xx/xx/2019 however fraudulent application have been submitted my id...,chase card wa reported on xx/xx/2019 however fraudulent application have been submitted my ident...
11,on xx/xx/2018 while trying to book a xxxx xxxx ticket i came across an offer for 300 00 t...,on xx/xx/2018 while trying to book a xxxx xxxx ticket i came across an offer for 300 00 to be ap...
14,my grand son give me check for 1600 00 i deposit it into my chase account after fund clear my...,my grand son give me check for 1600 00 i deposit it into my chase account after fund clear my ch...
...,...,...
78303,after being a chase card customer for well over a decade was offered multiple solicitations for...,after being a chase card customer for well over a decade wa offered multiple solicitation for ac...
78309,on wednesday xx/xx/xxxx i called chas my xxxx xxxx visa credit card provider and asked how to...,on wednesday xx/xx/xxxx i called chas my xxxx xxxx visa credit card provider and asked how to ma...
78310,i am not familiar with xxxx pay and did not understand the great risk this provides to consumers...,i am not familiar with xxxx pay and did not understand the great risk this provides to consumer ...
78311,i have had flawless credit for 30 yrs i ve had chase credit cards chase freedom specifica...,i have had flawless credit for 30 yr i ve had chase credit card chase freedom specifically since...


In [46]:
#Create a dataframe('df_clean') that will have only the complaints and the lemmatized complaints


In [47]:
df_clean = df[['complaint_what_happened_cleaned','complaint_what_happened_lemma']]

In [48]:
#text = df_clean['complaint_what_happened_lemma']

In [49]:
#Write your function to extract the POS tags
'''
def pos_tag(text):
  # write your code here



df_clean["complaint_POS_removed"] =  #this column should contain lemmatized text with all the words removed which have tags other than NN[tag == "NN"].
'''

'\ndef pos_tag(text):\n  # write your code here\n\n\n\ndf_clean["complaint_POS_removed"] =  #this column should contain lemmatized text with all the words removed which have tags other than NN[tag == "NN"].\n'

In [50]:
def extract_nouns_column(text):
    words = word_tokenize(text)  # Tokenize text
    tagged_words = pos_tag(words)  # Get POS tags
    lemmatized_nouns = [lemmatizer.lemmatize(word) for word, tag in tagged_words if tag == 'NN']  # Keep only nouns
    return ' '.join(lemmatized_nouns)  # Return as a single string

In [51]:
#tagged_words

In [ ]:
df['lemmatized_nouns'] = df['complaint_what_happened_lemma'].progress_apply(extract_nouns_column)

 41%|███████████████████████████████▉                                             | 8733/21072 [01:47<02:31, 81.34it/s]

In [ ]:
df.rename(columns={'lemmatized_nouns':'complaint_POS_removed'},inplace=True)

In [ ]:
df['complaint_POS_removed']

In [ ]:
#df

In [ ]:
#Write your function to extract the POS tags
'''
def pos_tag(text):
  # write your code here



df_clean["complaint_POS_removed"] =  #this column should contain lemmatized text with all the words removed which have tags other than NN[tag == "NN"].
'''

In [ ]:
#The clean dataframe should now contain the raw complaint, lemmatized complaint and the complaint after removing POS tags.
df_clean = df[['complaint_what_happened_cleaned','complaint_what_happened_lemma','complaint_POS_removed']]

In [ ]:
df_clean

## Exploratory data analysis to get familiar with the data.

Write the code in this task to perform the following:

*   Visualise the data according to the 'Complaint' character length
*   Using a word cloud find the top 40 words by frequency among all the articles after processing the text
*   Find the top unigrams,bigrams and trigrams by frequency among all the complaints after processing the text. ‘




In [ ]:
# Write your code here to visualise the data according to the 'Complaint' character length
df_clean['Char_length'] = [len(x) for x in df_clean['complaint_what_happened_cleaned']]

In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(data=df_clean,x=df_clean['Char_length'],bins=20)
plt.title('Distribution of Character Length')
plt.xlabel('Complaint Lenth')
plt.ylabel('No Of Complaints')
plt.show()

#### Find the top 40 words by frequency among all the articles after processing the text.

In [ ]:
from wordcloud import WordCloud, STOPWORDS
from collections import Counter

In [ ]:
'''
#df_clean['complaint_what_happened_cleaned']
all_text = ' '.join(df_clean['complaint_what_happened_cleaned'])
words = all_text.split()
word_counts = Counter(words)
#word_counts = dict(word_counts)
top_words = dict(word_counts.most_common(40))
'''

In [ ]:
'''
wordcloud = WordCloud(width=800, height=400, background_color="white").generate_from_frequencies(top_words)

# Display the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()
'''

In [ ]:
'''
# Add 'xx/xx/xxxx' to stopwords
custom_stopwords = STOPWORDS.union({'xx/xx/xxxx','xx','XXXX','x'})

wordcloud = WordCloud(
    stopwords=custom_stopwords,
    #width=800, 
    #height=400, 
    background_color="white",
    max_font_size=38,
    max_words=38,
    random_state=42
).generate_from_frequencies(top_words)

# Display the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()
'''

In [ ]:
'''
stop_words = set(STOPWORDS)
word_cloud = WordCloud(
    background_color='white',  # Change background color if needed
    stopwords=stop_words,  # Ensure stop_words is defined or set to None
    max_font_size=38,
    max_words=38,
    random_state=42
).generate(" ".join(df_clean['complaint_POS_removed']))  # Convert column to a single string

# Display the Word Cloud
plt.figure(figsize=(10, 5))
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis("off")
plt.show()
'''

In [ ]:
custom_stopwords = STOPWORDS.union({'xx/xx/xxxx','xx','xxxx'})
word_cloud = WordCloud(
    background_color='white',  # Change background color if needed
    stopwords=custom_stopwords,  # Ensure stop_words is defined or set to None
    max_font_size=38,
    max_words=38,
    random_state=42
).generate(" ".join(df_clean['complaint_POS_removed']))  # Convert column to a single string

# Display the Word Cloud
plt.figure(figsize=(10, 5))
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
'''
stop_words = set(STOPWORDS)
word_cloud = WordCloud(
                          background_color='green',
                          stopwords=stop_words,
                          max_font_size=38,
                          max_words=38,
                          random_state=42
                         ).generate(str(df_clean['complaint_POS_removed']))

fig = plt.figure(figsize=(20,16))
plt.imshow(word_cloud)
plt.axis('off')
plt.show()
'''

In [ ]:
#Using a word cloud find the top 40 words by frequency among all the articles after processing the text


In [ ]:
#Removing -PRON- from the text corpus
df_clean['Complaint_clean'] = df_clean['complaint_POS_removed'].str.replace('-PRON-', '')

#### Find the top unigrams,bigrams and trigrams by frequency among all the complaints after processing the text.

In [ ]:
#Write your code here to find the top 30 unigram frequency among the complaints in the cleaned datafram(df_clean).
df_clean['unigrams'] = df_clean['Complaint_clean'].apply(word_tokenize)

In [ ]:
all_unigrams = [word for words_list in df_clean['unigrams'] for word in words_list]
unigram_freq = Counter(all_unigrams)
# Get the top 30 unigrams
top_30_unigrams = unigram_freq.most_common(30)

# Convert to DataFrame for better visualization
df_top_unigrams = pd.DataFrame(top_30_unigrams, columns=['Unigram', 'Frequency'])

# Display the result
print(df_top_unigrams)

In [ ]:
from nltk.corpus import stopwords
# Download stopwords if not already downloaded
#nltk.download('stopwords')
# Load default English stopwords
stop_words = set(stopwords.words('english'))

# Add custom stopwords
custom_stopwords = {'xx', 'xxxx', 'xx/xx/xxxx', '00/00/0000', 'dummyword'}  # Add your custom words here
stop_words.update(custom_stopwords)  # Merge both sets

# Extract unigrams from each complaint and remove stopwords
df_clean['unigrams'] = df_clean['Complaint_clean'].progress_apply(
    lambda x: [word for word in word_tokenize(x) if word.lower() not in stop_words]
)

# Flatten the list of all unigrams across complaints
all_unigrams = [word for words_list in df_clean['unigrams'] for word in words_list]

# Count the frequency of each unigram
unigram_freq = Counter(all_unigrams)

# Get the top 30 unigrams
top_30_unigrams = unigram_freq.most_common(30)

# Convert to DataFrame for better visualization
df_top_unigrams = pd.DataFrame(top_30_unigrams, columns=['Unigram', 'Frequency'])

# Display the result
print(df_top_unigrams)

In [ ]:
#import pandas as pd
#import matplotlib.pyplot as plt
#from collections import Counter
#from nltk.tokenize import word_tokenize
#from nltk.corpus import stopwords
from nltk.util import bigrams, trigrams
#import nltk

# Download stopwords if not already downloaded
#nltk.download('stopwords')

# Load default English stopwords
stop_words = set(stopwords.words('english'))

# Add custom stopwords
custom_stopwords = {'XX', 'XXXX', 'xx/xx/xxxx', '00/00/0000', 'dummyword'}
stop_words.update(custom_stopwords)  # Merge both sets

# Function to generate bigrams and trigrams
def get_ngrams(text, ngram_type='bigram'):
    words = [word for word in word_tokenize(text) if word.lower() not in stop_words]
    if ngram_type == 'bigram':
        return list(bigrams(words))  # Generate bigrams
    elif ngram_type == 'trigram':
        return list(trigrams(words))  # Generate trigrams
    return []

# Extract bigrams and trigrams from each complaint
df_clean['bigrams'] = df_clean['Complaint_clean'].apply(lambda x: get_ngrams(x, 'bigram'))
df_clean['trigrams'] = df_clean['Complaint_clean'].apply(lambda x: get_ngrams(x, 'trigram'))

# Flatten the list of bigrams and trigrams across all complaints
all_bigrams = [pair for pairs_list in df_clean['bigrams'] for pair in pairs_list]
all_trigrams = [triplet for triplets_list in df_clean['trigrams'] for triplet in triplets_list]

# Count the frequency of each bigram and trigram
bigram_freq = Counter(all_bigrams)
trigram_freq = Counter(all_trigrams)

# Get the top 30 bigrams and trigrams
top_30_bigrams = bigram_freq.most_common(30)
top_30_trigrams = trigram_freq.most_common(30)

# Convert to DataFrame for visualization
df_top_bigrams = pd.DataFrame(top_30_bigrams, columns=['Bigram', 'Frequency'])
df_top_trigrams = pd.DataFrame(top_30_trigrams, columns=['Trigram', 'Frequency'])

# Display the top bigrams and trigrams
print("Top 30 Bigrams:\n", df_top_bigrams)
print("\nTop 30 Trigrams:\n", df_top_trigrams)

In [ ]:
#Print the top 10 words in the unigram frequency


In [ ]:
#Write your code here to find the top 30 bigram frequency among the complaints in the cleaned datafram(df_clean).


In [ ]:
#Print the top 10 words in the bigram frequency

In [ ]:
#Write your code here to find the top 30 trigram frequency among the complaints in the cleaned datafram(df_clean).


In [ ]:
#Print the top 10 words in the trigram frequency

## The personal details of customer has been masked in the dataset with xxxx. Let's remove the masked text as this will be of no use for our analysis

In [ ]:
df_clean['Complaint_clean'] = df_clean['Complaint_clean'].str.replace('xxxx','')

In [ ]:
#All masked texts has been removed
df_clean

## Feature Extraction
Convert the raw texts to a matrix of TF-IDF features

**max_df** is used for removing terms that appear too frequently, also known as "corpus-specific stop words"
max_df = 0.95 means "ignore terms that appear in more than 95% of the complaints"

**min_df** is used for removing terms that appear too infrequently
min_df = 2 means "ignore terms that appear in less than 2 complaints"

In [ ]:
#Write your code here to initialise the TfidfVectorizer
tfidf = TfidfVectorizer(max_df=0.95,min_df=2,stop_words='english')


#### Create a document term matrix using fit_transform

The contents of a document term matrix are tuples of (complaint_id,token_id) tf-idf score:
The tuples that are not there have a tf-idf score of 0

In [ ]:
#Write your code here to create the Document Term Matrix by transforming the complaints column present in df_clean.
dtm = tfidf.fit_transform(df_clean['Complaint_clean'])

In [ ]:
#print(tfidf_model)
print(tfidf_model.toarray())

In [ ]:
#vectorizer.get_feature_names_out()[:10]
pd.DataFrame(dtm.toarray(),columns=tfidf.get_feature_names_out())

## Topic Modelling using NMF

Non-Negative Matrix Factorization (NMF) is an unsupervised technique so there are no labeling of topics that the model will be trained on. The way it works is that, NMF decomposes (or factorizes) high-dimensional vectors into a lower-dimensional representation. These lower-dimensional vectors are non-negative which also means their coefficients are non-negative.

In this task you have to perform the following:

* Find the best number of clusters
* Apply the best number to create word clusters
* Inspect & validate the correction of each cluster wrt the complaints
* Correct the labels if needed
* Map the clusters to topics/cluster names

In [ ]:
from sklearn.decomposition import NMF

## Manual Topic Modeling
You need to do take the trial & error approach to find the best num of topics for your NMF model.

The only parameter that is required is the number of components i.e. the number of topics we want. This is the most crucial step in the whole topic modeling process and will greatly affect how good your final topics are.

In [ ]:
#Load your nmf_model with the n_components i.e 5
num_topics = 5 #write the value you want to test out

#keep the random_state =40
nmf_model = NMF(n_components=num_topics,random_state=40)



In [ ]:
W = nmf_model.fit_transform(dtm)  # Document-topic matrix
H = nmf_model.components_       # Topic-term matrix
len(tfidf.get_feature_names_out())

In [ ]:
words = np.array(tfidf.get_feature_names_out())
words

In [ ]:
'''
topic_words = pd.DataFrame(np.zeros((num_topics,15)),
             index=[f'Topic {i + 1}' for i in range(num_topics)],
             columns=[f'Word {i + 1}' for i in range(15)]).astype(str)

for i in range(num_topics):
    ix = H[i].argsort()[::-1][:15]
    topic_words.iloc[i] = words[ix]

topic_words
'''

In [ ]:
#Print the Top15 words for each of the topics

topic_words = pd.DataFrame(np.zeros((15,num_topics)),
             #columns=[f'Topic {i + 1}' for i in range(num_topics)],
             columns=[f'{i + 1}' for i in range(num_topics)],
             index=[f'Word {i + 1}' for i in range(15)]).astype(str)

for i in range(num_topics):
    ix = H[i].argsort()[::-1][:15]
    topic_words[f'{i + 1}'] = words[ix]

topic_words

In [ ]:
#Print the Top15 words for each of the topics
'''
words = np.array(vectorizer.get_feature_names_out)
topic_words = pd.DataFrame(np.zeros((num_topics, 10)), index=[f'Topic {i + 1}' for i in range(num_topics)],
                           columns=[f'Word {i + 1}' for i in range(10)]).astype(str)
for i in range(num_topics):
    ix = H[i].argsort()[::-1][:10]
    topic_words.iloc[i] = words[ix]

topic_words
'''

In [ ]:
'''
n_topics = 5  # Define number of topics
nmf_model = NMF(n_components=n_topics, random_state=42)  # Initialize NMF
W = nmf_model.fit_transform(tfidf_model)  # Document-topic matrix
H = nmf_model.components_  # Topic-word matrix

# Step 3: Extract Top 15 Words for Each Topic
feature_names = vectorizer.get_feature_names_out()

topics_dict = {}  # Dictionary to store topic words

for topic_idx, topic in enumerate(H):
    top_words = [feature_names[i] for i in topic.argsort()[:-16:-1]]  # Get top 15 words
    topics_dict[f"Topic {topic_idx+1}"] = top_words

# Convert dictionary to DataFrame
df_topics = pd.DataFrame(topics_dict)

df_topics
'''

In [ ]:
#Create the best topic for each complaint in terms of integer value 0,1,2,3 & 4
#W_df = pd.DataFrame(W, columns=[f'{i + 1}' for i in range(num_topics)])
#W_df['max_topic'] = W_df.apply(lambda x: x.idxmax(), axis=1)
#W_df

In [ ]:
#Assign the best topic to each of the cmplaints in Topic Column
df_clean['Topic'] = np.argmax(W,axis=1) + 1

In [ ]:
df_clean[['complaint_what_happened_cleaned','Complaint_clean','Topic']]

In [ ]:
#Print the first 5 Complaint for each of the Topics
#df_clean=df_clean.groupby('Topic').head(5)
#df_clean.sort_values('Topic')

df_clean[['complaint_what_happened_cleaned','Topic']].groupby('Topic').head(5).sort_values('Topic')

#### After evaluating the mapping, if the topics assigned are correct then assign these names to the relevant topic:
* Bank Account services
* Credit card or prepaid card
* Theft/Dispute Reporting
* Mortgage/Loan
* Others

In [ ]:
'''
#Create the dictionary of Topic names and Topics
Topic_names = {0: 'Bank Account services',
    1: 'Credit card or prepaid card',
    2: 'Others',
    3: 'Theft/Dispute Reporting',
    4: 'Mortgage/Loan'   }
#Replace Topics with Topic Names
df_clean['Topic'] = df_clean['Topic'].map(Topic_names)
'''

In [ ]:
#Create the dictionary of Topic names and Topics

Topic_names = {1: 'Bank Account services',
    2: 'Credit card or prepaid card',
    3: 'Others',
    4: 'Theft/Dispute Reporting',
    5: 'Mortgage/Loan'   }
#Replace Topics with Topic Names
df_clean['Topic'] = df_clean['Topic'].map(Topic_names)

In [ ]:
df_clean

In [ ]:
sns.countplot(data=df_clean,y=df_clean['Topic_Categorey'])
plt.show()

## Supervised model to predict any new complaints to the relevant Topics.

You have now build the model to create the topics for each complaints.Now in the below section you will use them to classify any new complaints.

Since you will be using supervised learning technique we have to convert the topic names to numbers(numpy arrays only understand numbers)

In [ ]:
df_clean.rename(columns={'Topic':'Topic_Categorey'},inplace=True)

In [ ]:
#Create the dictionary again of Topic names and Topics
Topic_Ids = {'Bank Account services' :1,
    'Credit card or prepaid card':2,
    'Others':3,
    'Theft/Dispute Reporting':4,
    'Mortgage/Loan':5    }
#Replace Topics with Topic Names
df_clean['Topic'] = df_clean['Topic_Categorey'].map(Topic_Ids)

In [ ]:
df_clean

In [ ]:
#Keep the columns"complaint_what_happened" & "Topic" only in the new dataframe --> training_data
training_data=df_clean[['complaint_what_happened_cleaned','Topic']]

In [ ]:
training_data

####Apply the supervised models on the training data created. In this process, you have to do the following:
* Create the vector counts using Count Vectoriser
* Transform the word vecotr to tf-idf
* Create the train & test data using the train_test_split on the tf-idf & topics


In [ ]:

#Write your code to get the Vector count


#Write your code here to transform the word vector to tf-idf

You have to try atleast 3 models on the train & test data from these options:
* Logistic regression
* Decision Tree
* Random Forest
* Naive Bayes (optional)

**Using the required evaluation metrics judge the tried models and select the ones performing the best**

In [ ]:
# Write your code here to build any 3 models and evaluate them using the required metrics



